In [2]:
!pip install folium geopandas branca

In [3]:
import pandas as pd
import geopandas as gpd
import folium
from shapely.geometry import LineString
from branca.colormap import linear
import requests

In [9]:
# ---------------- CONFIGURAÇÕES ----------------
estado = "RS"
titulo_mapa = "Layout de Mapas Temático Quantitativos de Linhas"

# ---------- DADOS SIMULADOS (LINHAS NO RS) ----------
dados = {
    'nome': ['Linha A', 'Linha B', 'Linha C', 'Linha D', 'Linha E'],
    'classe': [1, 2, 3, 4, 5],
    'coordenadas': [
        [(-51.2, -32.5), (-51.3, -29.6)],
        [(-52.5, -31.2), (-52.6, -30.3)],
        [(-53.4, -32.0), (-53.5, -31.1)],
        [(-54.0, -31.8), (-54.1, -30.9)],
        [(-52.0, -32.8), (-52.1, -29.9)],
    ]
}

df = pd.DataFrame(dados)
geometry = [LineString(coords) for coords in df['coordenadas']]
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")

# ---------- MAPA BASE ----------
mapa = folium.Map(location=[-30.0, -53.0], zoom_start=6, tiles="OpenStreetMap")

# ---------- CAMADA DE LINHAS ----------
linhas = folium.FeatureGroup(name="Linhas (Classe 1 a 5)")
cores = linear.Oranges_09.scale(1, 5)

for _, row in gdf.iterrows():
    classe = row['classe']
    cor = cores(classe)
    espessura = 1 + classe * 2
    coords = [(lat, lon) for lon, lat in row.geometry.coords]

    folium.PolyLine(
        locations=coords,
        color=cor,
        weight=espessura,
        popup=f"{row['nome']} - Classe: {classe}"
    ).add_to(linhas)

cores.caption = "Classe (1 a 5)"
cores.add_to(mapa)
linhas.add_to(mapa)

# ---------- CAMADA ESTADO ----------
url_estado = f"https://raw.githubusercontent.com/josecacciatore89/Estados_BR_IBGE/main/{estado}.geojson"
camada_estado = folium.FeatureGroup(name=f"Estado - {estado}")

try:
    geojson_estado = requests.get(url_estado).json()
    folium.GeoJson(
        geojson_estado,
        name=estado,
        style_function=lambda x: {
            "fillOpacity": 0.0,
            "color": "black",
            "weight": 2
        },
        highlight_function=lambda x: {"weight": 3, "color": "blue"},
        tooltip=estado
    ).add_to(camada_estado)
    camada_estado.add_to(mapa)
except Exception as e:
    print(f"Erro ao carregar estado {estado}: {e}")

# ---------- CAMADA MUNICÍPIOS ----------
url_municipios = f"https://raw.githubusercontent.com/josecacciatore89/municipios_BR_IBGE2023/main/{estado}_2023.geojson"
camada_municipios = folium.FeatureGroup(name=f"Municípios {estado}")

try:
    geojson_municipios = requests.get(url_municipios).json()
    folium.GeoJson(
        geojson_municipios,
        name=f"Municipios_{estado}",
        style_function=lambda x: {
            "fillOpacity": 0.0,
            "color": "black",
            "weight": 0.5
        },
        popup=folium.GeoJsonPopup(fields=["NM_MUN"]),
        tooltip=folium.GeoJsonTooltip(fields=["NM_MUN"])
    ).add_to(camada_municipios)
    camada_municipios.add_to(mapa)
except Exception as e:
    print(f"Erro ao carregar municípios do estado {estado}: {e}")

# ---------- CONTROLE DE CAMADAS ----------
folium.LayerControl(collapsed=False).add_to(mapa)

# ---------- CABEÇALHO COM TÍTULO ----------
cor_fundo = "#FFFFFF"
cor_fonte = "#000000"
html = f"""
<div style="background-color: {cor_fundo}; color: {cor_fonte}; font-family: Arial; padding: 20px; text-align: center; font-weight: bold; font-size: 20px;">
    {titulo_mapa}
</div>
"""
mapa.get_root().html.add_child(folium.Element(html))

# ---------- LOGOTIPOS ----------
left_logo = '<img src="https://raw.githubusercontent.com/josecacciatore89/imagens/main/cropped-logo.png" style="position:absolute;top:10px;left:100px;width:150px;">'
right_logo = '<img src="https://raw.githubusercontent.com/josecacciatore89/imagens/main/cropped-logo.png" style="position:absolute;top:10px;right:100px;width:150px;">'
mapa.get_root().html.add_child(folium.Element(left_logo))
mapa.get_root().html.add_child(folium.Element(right_logo))

# ---------- SALVA O MAPA ----------
mapa.save("mapa_linhas.html")
print("Mapa salvo como mapa_linhas.html")

Mapa salvo como mapa_linhas.html
